In [1]:
%load_ext autoreload
%autoreload 2
from os.path import join
from tqdm import tqdm
import pandas as pd
import numpy as np
import clin.eval.med_status
import datasets
import clin.parse
import openai
openai.api_key_path = '/home/chansingh/.OPENAI_KEY'
from typing import List
results_dir = '../results/'
from clin.config import PATH_REPO
import imodelsx.process_results
r = imodelsx.process_results.get_results_df(results_dir, use_cached=False)

def viz_blues(df):
    return df.style.format(precision=3).background_gradient(cmap='Blues')

100%|██████████| 24/24 [00:00<00:00, 1673.96it/s]


# Average results

In [2]:
# medication_status
r_med = r[r.dataset_name == 'medication_status']
df = pd.DataFrame.from_dict(datasets.load_dataset('mitclinicalml/clinical-ie', 'medication_status')['test'])
r_med = clin.eval.med_status.add_status_eval(r_med, df)
r_med = imodelsx.process_results.average_over_seeds(r_med, experiment_filename=join(PATH_REPO, 'experiments', 'eval_model.py'))

# compare values for a single row
row_df = r_med[(r_med.n_shots == 5) * (r_med.checkpoint == 'text-davinci-003')].reset_index()
rc = row_df[[c for c in row_df.columns if '___' in c]]
# create multindex columns by splitting on '___'
rc = rc.rename(columns={c: tuple(c.split('___')) for c in rc.columns})

# convert tuple column names to multiindex
rc.columns = pd.MultiIndex.from_tuples(rc.columns)
rc = rc.T.reset_index()
rc = rc.rename(columns={
    'level_0': '',
    'level_1': 'Verifiers',
}).pivot_table(index='Verifiers', columns='', values=0).round(3)
# rc.style.format(precision=3).background_gradient(cmap='gray')

cols = {
    'f1': 'F1 (Med)',
    'precision': 'Precision (Med)',
    'recall': 'Recall (Med)',
    'status_f1_macro_cond': 'F1 (Med status)',
}
rows = {
    'original': 'Original',
    'ov': 'Omission',
    'pv': 'Prune',
    'ov_pv': 'Omission + Prune',
    'sv': 'Omission + Prune + Evidence'
}
rt_med = rc[list(cols.keys())].rename(columns=cols).loc[list(rows.keys())].rename(index=rows)
viz_blues(rt_med)

  0%|          | 0/2 [00:00<?, ?it/s]

/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r[k] = d[k]
/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  r[k] = d[k]
/home/chansingh/clinical-extract-and-check/clin/eval/med_status.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentati

,F1 (Med),Precision (Med),Recall (Med),F1 (Med status)
Verifiers,,,,
Original,0.930,0.933,0.927,0.626
Omission,0.919,0.891,0.948,0.625
Prune,0.933,0.950,0.917,0.626
Omission + Prune,0.938,0.945,0.931,0.625
Omission + Prune + Evidence,0.937,0.946,0.928,0.648


In [3]:
r_ebm = r[r.dataset_name == "ebm"]
row = r_ebm[(r_ebm.n_shots == 5) * (r_ebm.checkpoint == "text-davinci-003")].iloc[0]

# show metrics
row_df = pd.DataFrame(
    pd.Series({k: row[k] for k in row.keys() if "___" in k}).round(3)
).T
rc = row_df[[c for c in row_df.columns if "___" in c]]
# create multindex columns by splitting on '___'
rc = rc.rename(columns={c: tuple(c.split("___")) for c in rc.columns})

# convert tuple column names to multiindex
rc.columns = pd.MultiIndex.from_tuples(rc.columns)
rc = rc.T.reset_index()
rc = (
    rc.rename(
        columns={
            "level_0": "",
            "level_1": "Verifiers",
        }
    )
    .pivot_table(index="Verifiers", columns="", values=0)
    .round(3)
)
# rc.index = [x.replace("list_", "") for x in rc.index.values]
cols = {
    "f1": "F1 (EBM)",
    "precision": "Precision (EBM)",
    "recall": "Recall (EBM)",
}
rows = {
    "original": "Original",
    "ov": "Omission",
    "pv": "Prune",
    "ov_pv": "Omission + Prune",
    "ov_pv_ev": "Omission + Prune + Evidence",
}
rt_ebm = rc[list(cols.keys())].rename(columns=cols).loc[list(rows.keys())].rename(index=rows)
viz_blues(rt_ebm)

,F1 (EBM),Precision (EBM),Recall (EBM)
Verifiers,,,
Original,0.506,0.489,0.523
Omission,0.505,0.442,0.589
Prune,0.580,0.589,0.570
Omission + Prune,0.574,0.530,0.626
Omission + Prune + Evidence,0.590,0.570,0.612


# Visualize

In [4]:
display(viz_blues(rt_med))
display(viz_blues(rt_ebm))

,F1 (Med),Precision (Med),Recall (Med),F1 (Med status)
Verifiers,,,,
Original,0.930,0.933,0.927,0.626
Omission,0.919,0.891,0.948,0.625
Prune,0.933,0.950,0.917,0.626
Omission + Prune,0.938,0.945,0.931,0.625
Omission + Prune + Evidence,0.937,0.946,0.928,0.648


,F1 (EBM),Precision (EBM),Recall (EBM)
Verifiers,,,
Original,0.506,0.489,0.523
Omission,0.505,0.442,0.589
Prune,0.580,0.589,0.570
Omission + Prune,0.574,0.530,0.626
Omission + Prune + Evidence,0.590,0.570,0.612


In [5]:
# add columns from rt_ebm to rt_med
rt = rt_med.join(rt_ebm, rsuffix=' (EBM)')
display(viz_blues(rt))

,F1 (Med),Precision (Med),Recall (Med),F1 (Med status),F1 (EBM),Precision (EBM),Recall (EBM)
Verifiers,,,,,,,
Original,0.930,0.933,0.927,0.626,0.506,0.489,0.523
Omission,0.919,0.891,0.948,0.625,0.505,0.442,0.589
Prune,0.933,0.950,0.917,0.626,0.580,0.589,0.570
Omission + Prune,0.938,0.945,0.931,0.625,0.574,0.530,0.626
Omission + Prune + Evidence,0.937,0.946,0.928,0.648,0.590,0.570,0.612
